In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV



In [3]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Severity Id,Issue Type Name,Threat Class,Security Risk,Cause,Analysis Result,Result
0,0,3,authentication credentials unprotected,insufficient authentication,it may be possible to bypass the web applicati...,insufficient authentication method was used by...,no need to fix,False Negative
1,1,3,bufferoverflow,buffer overflow,it is possible to prevent the web application ...,proper bounds checking were not performed on i...,no need to fix,False Negative
2,2,3,command injection,os commanding,it is possible to run remote commands on the w...,no validation was done in order to make sure t...,no need to fix,False Negative
3,3,3,cryptography insecurealgorithm,information leakage,it is possible to gather sensitive information...,sensitive information stored to disk in cleartext,no need to fix,False Negative
4,4,3,improper handling of exceptional conditions,information leakage,it is possible to retrieve the absolute path o...,exceptions and error messages which may conta...,no need to fix,False Negative


In [5]:
# preprocess the dataset

data.fillna('', inplace=True)
X_text = data.drop(columns=['Unnamed: 0', 'Severity Id', 'Analysis Result', 'Result'])
X_numerical = data[['Severity Id']]
y = data[['Analysis Result', 'Result']]

In [6]:
# One-hot encode 'Severity Id'
encoder = OneHotEncoder()
X_numerical_encoded = encoder.fit_transform(X_numerical)

In [7]:
# Convert text features to numerical representations
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_text_tfidf = tfidf_vectorizer.fit_transform(X_text['Issue Type Name'] + ' ' + X_text['Cause'] + ' ' + X_text['Threat Class'] + ' ' + X_text['Security Risk'])


In [8]:
# Concatenate text and numerical features
X_combined = pd.concat([pd.DataFrame(X_numerical_encoded.toarray()), pd.DataFrame(X_text_tfidf.toarray())], axis=1)


In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


In [10]:
# Use the best hyperparameters to train the model
best_rf_classifier = RandomForestClassifier(random_state=42, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50)
multioutput_classifier_tuned = MultiOutputClassifier(best_rf_classifier)
multioutput_classifier_tuned.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=50,
                                                       random_state=42))

In [11]:
# Make predictions using the tuned model
y_pred_tuned = multioutput_classifier_tuned.predict(X_test)

In [12]:
# Evaluate the tuned model
accuracy_analysis_result_tuned = accuracy_score(y_test['Analysis Result'], y_pred_tuned[:, 0])
print("Accuracy for Analysis Result (Tuned):", accuracy_analysis_result_tuned)

Accuracy for Analysis Result (Tuned): 0.7658536585365854


In [13]:
accuracy_result_tuned = accuracy_score(y_test['Result'], y_pred_tuned[:, 1])
print("Accuracy for Result (Tuned):", accuracy_result_tuned)

Accuracy for Result (Tuned): 0.9609756097560975


In [13]:
# Print classification report for both outputs
print("Classification Report for Analysis Result:")
print(classification_report(y_test['Analysis Result'], y_pred_tuned[:, 0]))

print("Classification Report for Result:")
print(classification_report(y_test['Result'], y_pred_tuned[:, 1]))

Classification Report for Analysis Result:
                precision    recall  f1-score   support

         fixed       0.00      0.00      0.00         1
   need to fix       0.00      0.00      0.00        45
no need to fix       0.77      0.99      0.87       159

      accuracy                           0.77       205
     macro avg       0.26      0.33      0.29       205
  weighted avg       0.60      0.77      0.67       205

Classification Report for Result:
                precision    recall  f1-score   support

     Duplicate       0.97      1.00      0.98       197
False Negative       0.00      0.00      0.00         2
False Positive       0.00      0.00      0.00         3
 True Positive       0.00      0.00      0.00         3

      accuracy                           0.96       205
     macro avg       0.24      0.25      0.25       205
  weighted avg       0.93      0.96      0.95       205



/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/en

In [14]:
# Print multilabel confusion matrices for both outputs
print("Multilabel Confusion Matrix for Analysis Result:")
print(multilabel_confusion_matrix(y_test['Analysis Result'], y_pred_tuned[:, 0]))

print("Multilabel Confusion Matrix for Result:")
print(multilabel_confusion_matrix(y_test['Result'], y_pred_tuned[:, 1]))

Multilabel Confusion Matrix for Analysis Result:
[[[204   0]
  [  1   0]]

 [[158   2]
  [ 45   0]]

 [[  0  46]
  [  2 157]]]
Multilabel Confusion Matrix for Result:
[[[  2   6]
  [  0 197]]

 [[203   0]
  [  2   0]]

 [[201   1]
  [  3   0]]

 [[201   1]
  [  3   0]]]


In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
# Preprocessing of dataset
new_data.fillna('', inplace=True)
X_new_text = new_data.drop(columns=['Unnamed: 0', 'Severity Id'])  # Exclude the output columns if present
X_new_numerical = new_data[['Severity Id']]

# Load the previously fitted encoder and transform the numerical features of the new data
X_new_numerical_encoded = encoder.fit_transform(X_new_numerical)

# Load the previously fitted TF-IDF vectorizer and transform the text features of the new data
X_new_text_tfidf = tfidf_vectorizer.transform(X_new_text['Issue Type Name'] + ' ' + X_new_text['Cause'] + ' ' + X_new_text['Threat Class'] + ' ' + X_new_text['Security Risk'])

# Concatenate text and numerical features for the new data
X_new_combined = pd.concat([pd.DataFrame(X_new_numerical_encoded.toarray()), pd.DataFrame(X_new_text_tfidf.toarray())], axis=1)

# Use the already trained classifier to predict the outputs for the new data
y_new_pred = multioutput_classifier_tuned.predict(X_new_combined)

# Get the predictions for Analysis Result and Result columns
y_new_analysis_result_pred = y_new_pred[:, 0]
y_new_result_pred = y_new_pred[:, 1]

new_data['Analysis Result Prediction'] = y_new_analysis_result_pred
new_data['Result Prediction'] = y_new_result_pred
print(new_data)

# Save the new_data DataFrame with predictions to a new CSV file
new_data.to_csv("new_data_with_predictions.csv", index=False)



    Unnamed: 0  Severity Id      Issue Type Name  \
0         1010            3        sql injection   
1         1011            3        sql injection   
2         1012            3  validation required   
3         1013            3  validation required   
4         1014            3  validation required   
5         1015            3  validation required   
6         1016            3  validation required   
7         1017            3        xml injection   
8         1018            3        xml injection   
9         1019            3        xml injection   
10        1020            3        xml injection   

                       Threat Class  \
0                     sql injection   
1                     sql injection   
2   insufficient process validation   
3   insufficient process validation   
4   insufficient process validation   
5   insufficient process validation   
6   insufficient process validation   
7                     xml injection   
8                     xm

In [25]:
import joblib
joblib.dump(encoder, 'encoder.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(multioutput_classifier_tuned, 'multioutput_classifier_tuned.p')

['multioutput_classifier_tuned.p']